In [11]:
import os
import torch
import clip
import numpy as np
import pandas as pd
import pyarrow.feather as feather
from PIL import Image
from urllib import request
from tqdm import tqdm
from torch.utils.data import Dataset, DataLoader
from torchvision import datasets, transforms

In [12]:
device = "cuda" if torch.cuda.is_available() else "cpu"
model, preprocess = clip.load("ViT-B/32", device=device)
device

'cpu'

In [13]:
id_data = feather.read_feather('/Users/hanselblanco/Documents/tesis/datasets/unsplash-research-dataset-lite-latest/data/id_data.feather')
id_data.head()

,photo_id,photo_url,photo_image_url,photo_submitted_at,photo_featured,photo_width,photo_height,photo_aspect_ratio,photo_description,photographer_username,...,photo_location_country,photo_location_city,stats_views,stats_downloads,ai_description,ai_primary_landmark_name,ai_primary_landmark_latitude,ai_primary_landmark_longitude,ai_primary_landmark_confidence,blur_hash
0,XMyPniM9LF0,https://unsplash.com/photos/XMyPniM9LF0,https://images.unsplash.com/uploads/1411949294...,2014-09-29 00:08:38.594364,t,4272,2848,1.50,Woman exploring a forest,michellespencer77,...,None,None,2375421,6967,woman walking in the middle of forest,None,NaN,NaN,NaN,L56bVcRRIWMh.gVunlS4SMbsRRxr
1,rDLBArZUl1c,https://unsplash.com/photos/rDLBArZUl1c,https://images.unsplash.com/photo-141633941111...,2014-11-18 19:36:57.08945,t,3000,4000,0.75,Succulents in a terrarium,ugmonk,...,None,None,13784815,82141,succulent plants in clear glass terrarium,None,NaN,NaN,NaN,LvI$4txu%2s:_4t6WUj]xat7RPoe
2,cNDGZ2sQ3Bo,https://unsplash.com/photos/cNDGZ2sQ3Bo,https://images.unsplash.com/photo-142014251503...,2015-01-01 20:02:02.097036,t,2564,1710,1.50,Rural winter mountainside,johnprice,...,None,None,1302461,3428,rocky mountain under gray sky at daytime,None,NaN,NaN,NaN,LhMj%NxvM{t7_4t7aeoM%2M{ozj[
3,iuZ_D1eoq9k,https://unsplash.com/photos/iuZ_D1eoq9k,https://images.unsplash.com/photo-141487280988...,2014-11-01 20:15:13.410073,t,2912,4368,0.67,Poppy seeds and flowers,krisatomic,...,None,None,2890238,33704,red common poppy flower selective focus phography,None,NaN,NaN,NaN,LSC7DirZAsX7}Br@GEWWmnoLWCnj
4,BeD3vjQ8SI0,https://unsplash.com/photos/BeD3vjQ8SI0,https://images.unsplash.com/photo-141700759404...,2014-11-26 13:13:50.134383,t,4896,3264,1.50,Silhouette near dark trees,jonaseriksson,...,None,None,8704860,49662,trees during night time,None,NaN,NaN,NaN,L25|_:V@0hxtI=W;odae0ht6=^NG


In [14]:
labels = ['morning','noon','afternoon','night','sunrise or sunset']
tkns = ['A photo taken at '+ label for label in labels]
text = clip.tokenize(tkns).to(device)

In [15]:
BATCH_SIZE = 100000
ln = len(id_data)

In [16]:
results = []

for i in tqdm(range(0, ln, BATCH_SIZE)):
    images = [preprocess(Image.open(f"/Users/hanselblanco/Documents/tesis/datasets/unsplash-research-dataset-lite-latest/images/{id_data['photo_id'][j]}.jpg")) for j in range(4)]
    image_input = torch.tensor(np.stack(images)).to(device)
    with torch.no_grad():
        image_features = model.encode_image(image_input)
        logits_per_image, logits_per_text = model(image_input, text)
        # The softmax function takes the original confidence and applys a transform to make all the confidence add up to one
        probs = logits_per_image.softmax(dim=-1).cpu().numpy()
        results.append(probs)

100%|██████████| 1/1 [00:00<00:00,  3.36it/s]


In [17]:
res = np.concatenate(results,axis=0)
choices = np.argmax(res,axis=1)
choices.shape

(4,)

In [18]:
getlabel = lambda x:labels[x]
vgetlabel = np.vectorize(getlabel)
times = vgetlabel(choices)
times

array(['afternoon', 'morning', 'noon', 'morning'], dtype='<U9')